In [1]:
# Import librerías python
import sys
import os
import jsonpickle

# Import de modulos ipynb
import import_ipynb
import environ
import bdoper

sys.path.append(os.getenv('MCT_PATH')) 

from funcRSA import setRSAkey, serializeKey, deSerializeKey
from classHash import hashClass

importing Jupyter notebook from environ.ipynb
importing Jupyter notebook from bdoper.ipynb


In [2]:
# Generar par de claves
key = setRSAkey(2048)
pubkey = key.publickey()

lista_nombres = ["Alberto Perez",
                 "Juan Garcia",
                 "Ana Junquera",
                 "Pedro Garcia",
                 "Emilia Gutierrez",
                 "Francisco Herrador"
                ]

bdoper.deleteTable('tableusers')
for nombre in lista_nombres:
    # Instanciamos la clase hash
    c_hash = hashClass(nombre, 256)

    # Firmamos el hash con la clave privada obtenida
    signed_hash = c_hash.signHash(key)

    # Serializamos la instancia para poder guardarla en BBDD
    clase_serializada =  jsonpickle.encode(c_hash, keys=True)
    
    # INSERT en bbdd
    bdoper.insert_user(nombre, serializeKey(pubkey), clase_serializada, signed_hash)


Alberto Perez añadido a la tabla
Juan Garcia añadido a la tabla
Ana Junquera añadido a la tabla
Pedro Garcia añadido a la tabla
Emilia Gutierrez añadido a la tabla
Francisco Herrador añadido a la tabla


In [3]:
# Comprobación de firma
# Tenemos en base de datos la clave publica y la firma
# Recuperamos la hashClass (columna 3) del usuario y lo des-serializamos 
# para poder obtener el self.hash que fue firmado

def checkSignedHash():
    for record in bdoper.select_all_user():
        user_class = jsonpickle.decode(record[3])
        rsa_key = deSerializeKey(record[2])
        signed_hash = record[4]
        firma_ok = user_class.verifySign(rsa_key,signed_hash)
        print(f'id: {record[0]} - {record[1]} - Verificación firma: {firma_ok}')

# Vamos a engañar al sistema para poner una firma falsa
# Generamos un nueva clave y hacemos un UPDATE del signed_hash de un usuario
def updateSignedHash(id_user):
    fake_key = setRSAkey(2048)
    fake_hash = hashClass('Anonimous', 256)
    fake_signed = fake_hash.signHash(fake_key)
    bdoper.upadate_sign(id_user, fake_signed)

In [4]:
checkSignedHash()

id: 106 - Alberto Perez - Verificación firma: True
id: 107 - Juan Garcia - Verificación firma: True
id: 108 - Ana Junquera - Verificación firma: True
id: 109 - Pedro Garcia - Verificación firma: True
id: 110 - Emilia Gutierrez - Verificación firma: True
id: 111 - Francisco Herrador - Verificación firma: True


In [5]:
updateSignedHash(110)

Falseada la firma del hash de 110


In [6]:
checkSignedHash()

id: 106 - Alberto Perez - Verificación firma: True
id: 107 - Juan Garcia - Verificación firma: True
id: 108 - Ana Junquera - Verificación firma: True
id: 109 - Pedro Garcia - Verificación firma: True
id: 110 - Emilia Gutierrez - Verificación firma: False
id: 111 - Francisco Herrador - Verificación firma: True
